## CAP/DVR step 1

In [1]:
import numpy as np
import cmath
from numpy.linalg import eig, eigvals
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
import pandas as pd

imports from local libs

In [2]:
import sys
sys.path.append('../../Python_libs')
import dvr
from jolanta import Jolanta_3D
import captools as ct

In [3]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [4]:
#
# Jolanata-3D parameters a, b, c: (0.028, 1.0, 0.028)
#
#   CS-DVR:   
#      bound state:  -7.17051 eV
#      resonance (3.1729556 - 0.16085j) eV
#
jparam=(0.028, 1.0, 0.028)

In [5]:
def Vcap(x, xcut=10):
    xabs = abs(x)
    if xabs < xcut:
        return 0
    else:
        return (xabs-xcut)**2

In [6]:
#
#  compute DVR of T, V, and W
#  then show the density of states
#  in a potential + energy-levels plot
#
xmax=30
xmin=0   # grid from -xmin to xmax
thresh = 7   # maximum energy for state in the plot in eV
ppB = 15     # grid points per Bohr
xcut=15

nGrid=int((xmax-xmin)*ppB)
print("nGrid = %d" % nGrid)
xs = dvr.DVRGrid(xmin, xmax, nGrid)
Vs = Jolanta_3D(xs, jparam)
Ws=np.array([Vcap(xs[i], xcut=xcut) for i in range(nGrid)])
Ts = dvr.KineticEnergy(1, xmin, xmax, nGrid)
[energy, wf] = dvr.DVRDiag2(nGrid, Ts, Vs, wf=True)

n_ene=0
for i in range(nGrid):
    print("%3d  %12.8f au = %12.5f eV" % (i+1, energy[i], energy[i]*au2eV))
    n_ene += 1
    if energy[i]*au2eV > thresh:
        break

# "DVR normalization", sum(wf[:,0]**2)
# this is correct for plotting

c=["orange", "blue"]
#h=float(xmax) / (nGrid+1.0)
scale=400

eta=2e-3
plt.cla()
plt.plot(xs,Vs*au2eV, '-', color="black")
plt.plot(xs,Ws*au2eV*eta, '-', color="green")
for i in range(n_ene):
    plt.plot(xs, scale*wf[:,i]**2+energy[i]*au2eV, '-', color=c[i%len(c)])
plt.ylim(energy[0]*au2eV-1, energy[n_ene-1]*au2eV+1)
plt.xlabel('$x$ [Bohr]')
plt.ylabel('$E$ [eV]')
plt.show()

nGrid = 450
  1   -0.26351131 au =     -7.17051 eV
  2    0.01683168 au =      0.45801 eV
  3    0.05606363 au =      1.52557 eV
  4    0.10555774 au =      2.87237 eV
  5    0.12662372 au =      3.44561 eV
  6    0.18685288 au =      5.08453 eV
  7    0.26221360 au =      7.13520 eV


In [9]:
""" complex diagonalization example """

eta=8e-4
print("eta = %f" % (eta))
H_eta = Ts + np.diag(Vs) - 1j*eta*np.diag(Ws)
energies = eigvals(H_eta)
energies.sort()
plt.cla()
plt.plot(energies.real*au2eV, energies.imag*au2eV, 'o')
plt.xlim(-1,10)
plt.ylim(-1.0,0.1)
plt.show()
energies[:30]*au2eV

eta = 0.000800


array([ -7.17050798-6.19764584e-11j,   0.94632115-5.18420571e-01j,
         1.64676727-1.55889259e+00j,   3.00538087-9.20129542e-01j,
         3.18658026-1.74608787e-01j,   5.02009331-8.97099869e-01j,
         7.09095443-8.33952308e-01j,   9.33990877-8.07634906e-01j,
        11.87330385-8.18257153e-01j,  14.74464007-8.27690189e-01j,
        17.94083962-8.30921863e-01j,  21.4507634 -8.31466353e-01j,
        25.26926172-8.31346273e-01j,  29.39500087-8.30903506e-01j,
        33.82619855-8.30272233e-01j,  38.56191119-8.29575160e-01j,
        43.60094213-8.28815644e-01j,  48.94264849-8.28095781e-01j,
        54.58626337-8.27365041e-01j,  60.53135113-8.26704600e-01j,
        66.77740071-8.26050839e-01j,  73.32410857-8.25470838e-01j,
        80.17112384-8.24902555e-01j,  87.31823074-8.24402169e-01j,
        94.76518248-8.23913813e-01j, 102.51182294-8.23484882e-01j,
       110.55797482-8.23066635e-01j, 118.90352331-8.22699289e-01j,
       127.54833868-8.22340854e-01j, 136.49233521-8.22025640e-

In [10]:
#
#  eta loop: 
#

n_keep=25
log_eta_min=-5
log_eta_max=-1
n_eta=25*(log_eta_max - log_eta_min)+1
etas=np.logspace(log_eta_min, log_eta_max, num=n_eta)

erdata = np.zeros((n_eta,n_keep), complex)  # array used to collect all theta-run data

for i_eta in range(n_eta):
    eta=etas[i_eta]
    H_eta = Ts + np.diag(Vs) - 1j*eta*np.diag(Ws)
    energies = eigvals(H_eta)
    energies.sort()
    erdata[i_eta,:] = energies[0:n_keep]
    print(i_eta+1, end=" ")
    if (i_eta+1)%10==0:
        print()

erdata *= au2eV

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 

In [12]:
#
# useful piece of the complex plane 
# (if unknown, plot all and zoom with matplotlib)
#
plt.cla()
for i in range(0, n_keep):
    plt.plot(erdata[:,i].real,  erdata[:,i].imag, 'o')
plt.xlim(0,8)
plt.ylim(-1,0)
plt.show()

In [13]:
#
#  get trajectory
#
follow=3.4  # for xmax=25 or 30
#follow=3.1
es=np.zeros(n_eta,complex)

for j in range(0,n_eta):
    i = np.argmin(abs(erdata[j,:]-follow))
    es[j] = erdata[j,i]
    follow = es[j]
plt.cla()
plt.plot(es.real, es.imag, 'o-')
plt.show()

In [14]:
df = pd.DataFrame({"eta": etas, "ReE":es.real, "ImE":es.imag})
df.to_csv('Traj_DVR_30.csv', index=False)
df.head(5)

,eta,ReE,ImE
0,0.000010,3.445541,-0.006558
1,0.000011,3.445528,-0.007191
2,0.000012,3.445512,-0.007884
3,0.000013,3.445493,-0.008645
4,0.000014,3.445470,-0.009479


In [ ]:
#
#  compute first and second derivative
#

corrs, absd1, absd2 = ct.trajectory_derivatives(etas, es)

plt.cla()
plt.figure(1)
plt.plot(es.real, es.imag, 'o-', color="blue")
plt.plot(corrs.real, corrs.imag, 'o-', color="brown")
plt.figure(2)
plt.plot(etas, absd1, 'o-', color="blue")
plt.plot(etas, absd2, 'o-', color="brown")
plt.xscale("log")
plt.show()

In [ ]:
fm0 = "{0:2d} {1:8.2e} {2:8.6f},{3:8.6f} {4:8.2e}  {5:8.6f},{6:8.6f} {7:8.2e}"
fm1 = "{0:2d} {1:8.2e} {2:8.6f},{3:8.6f} {4:8.2e}* {5:8.6f},{6:8.6f} {7:8.2e}"
fm2 = "{0:2d} {1:8.2e} {2:8.6f},{3:8.6f} {4:8.2e}  {5:8.6f},{6:8.6f} {7:8.2e}*"
fm3 = "{0:2d} {1:8.2e} {2:8.6f},{3:8.6f} {4:8.2e}* {5:8.6f},{6:8.6f} {7:8.2e}*"


print(fm0.format(0, etas[i], es[i].real, es[i].imag, absd1[i], corrs[i].real, corrs[i].imag, absd2[i]))
for i in range(1, n_eta):
    der1_neg = (absd1[i]-absd1[i-1] < 0)
    der2_neg = (absd2[i]-absd2[i-1] < 0)
    if der1_neg and der2_neg:
        frmt = fm3
    elif der1_neg:
        frmt = fm1
    elif der2_neg:
        frmt = fm2
    else:
        frmt = fm0
    print(frmt.format(i, etas[i], es[i].real, es[i].imag, absd1[i], corrs[i].real, corrs[i].imag, absd2[i]))

### CAP-results

$r_{cut} = 15$ Bohr

Complex stabilization:

 |         | $E_r$      | $E_i$    | $\eta_{opt}$ | abs(der) |
 | :---    | ---        | ---      | ---          | --- |
 | CS      | 3.1729556  | -0.16085 |          $-$ | $-$ |
 | CAP0-25 | 3.177      | -0.131   |  3.3e-03 | 1.1e-02 |
 | CAP1-25 | 3.146      | -0.131   |  3.3e-03 | 1.9e-02 |
 | CAP0-30 | 3.168      | -0.155   |  1.7e-03 | 2.4e-02 | 
 | CAP1-30 | 3.195      | -0.176   |  1.3e-03 | 5.5e-02 |
 | CAP0-35 | 3.173      | -0.161   |  7.6e-04 | 2.1e-03 |
 | CAP1-35 | 3.174      | -0.160   |  9.1e-04 | 1.6e-02 | 

| Remarks about complex trajectory |
| --- |
| $r_{max}=25$ Bohr: one sharp minimum |
| $r_{max}=30$ Bohr: long flat stretches |
| $r_{max}=35$ Bohr: one sharp minimium much lower than 25 or 30 |

Real stabilization needs to bedone for R0 and R1. Use GTO boxes.

|         | $E_r$      | $E_i$    | $\eta_{opt}$ | $\eta_{opt}^\prime$ |
| :---    | ---        | ---      | ---          | --- |
| CS      | 3.1729556  | -0.16085 |          $-$ | $-$ |
| CAP0-25 | 3.177      | -0.125   | 3.3e-3  | 7.6e-3 |
| CAP1-25 | 3.146      | -0.109   | 5.8e-3  | 2.1e-2 |
| CAP0-30 | 3.167      | -0.125   | 2.1e-3  | 7.6e-3 |
| CAP1-30 | 3.144      | -0.175   | 4.4e-3  | 1.4e-3 |
|         | 3.144      | -0.109   | 4.4e-3  | 2.1e-2 |
| CAP0-35 | 3.166      | -0.160   | 2.1e-3  | 8.3e-4 | 
|         | 3.166      | -0.125   | 2.1e-3  | 7.6e-3 | 
| CAP1-35 | 3.012      | -0.160   | 2.1e-4  | 9.1e-4 | 
|         | 3.144      | -0.109   | 4.4e-3  | 2.1e-2 | 


Seems for a short absorbance region, there is not much difference. But for a longer absorbance region, stabilization of the complex trajectory works better. 

In [ ]:
ct.JZBEK_plot(etas, corrs)
print("min(Re(Es)):", ct.find_local_minima(etas, corrs.real))
print("max(Im(Es)):", ct.find_local_maxima(etas, corrs.imag))

### Now get the complex wavefunction for plotting

In [ ]:
eta=8.04e-04 # eta_opt for xmax=35, xcut=15
Eres=3.17-1j*0.16
H_eta = Ts + np.diag(Vs) - 1j*eta*np.diag(Ws)
energies, wfs = eig(H_eta)
energies*=au2eV
i_res=np.argmin(abs(energies-Eres))
print(energies[i_res])
wf=wfs[:,i_res]

In [ ]:
plt.cla()
plt.figure(1)
plt.plot(xs, wf.real, '-')
plt.plot(xs, wf.imag, '-')
plt.figure(2)
plt.plot(xs, 20*abs(wf), '-')
plt.plot(xs, np.angle(wf), '-')
plt.show()